In [1]:
import numpy as np
import torch
import torch.utils.data as data
import pandas as pd
from src.error_measures import get_error_measures
from src.models import TwoLayerPerceptron, TwoLayerLeaky, TwoLayerSynaptic, ThreeLayerLeaky
from src.train_model import training_loop

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [3]:
np.random.seed(373)
torch.manual_seed(373)

In [4]:
data06 = pd.read_csv("../data/processed/target06.csv")
data06.drop(columns=["weekday"], inplace=True)

In [5]:
all_data = data.TensorDataset(torch.from_numpy((data06.values[:,:-1] - data06.values[:,:-1].min(0)) / data06.values[:,:-1].ptp(0)).float(),
                              torch.from_numpy(data06.values[:,-1]).float())  # normalization
train_dataset, test_dataset = torch.utils.data.random_split(all_data, (round(0.4 * len(all_data)), round(0.6 * len(all_data))))
train_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=256, shuffle=False, drop_last=False)

### Two layer perceptron

In [6]:
for i in range(5):
    h = np.random.randint(500, 3000)
    e = np.random.randint(20, 200)
    lr = np.random.uniform(0.0001, 0.005)
    print(f'\n------------- Random set: {i} -----------------\nHidden: {h}, epochs: {e}, learning rate: {lr}')
    model = TwoLayerPerceptron(num_inputs=8, num_hidden=h, num_outputs=1)
    training_loop(model, train_loader, test_loader, device, num_epochs=e, lr=lr)
    get_error_measures(model, test_loader, device=device, print_e=True)


------------- Random set: 0 -----------------
Hidden: 566, epochs: 29, learning rate: 0.004242469415092643


100%|██████████| 29/29 [00:12<00:00,  2.31it/s]


Accuracy on validation dataset: 36.60181082643036%
MAE: 7.206813714564409, MSE:93.38832492106202,
RMSE: 9.663763496747114, Index of Agreement: 0.8796608876895443

------------- Random set: 1 -----------------
Hidden: 1080, epochs: 168, learning rate: 0.0038237463340484675


100%|██████████| 168/168 [00:44<00:00,  3.78it/s]


Accuracy on validation dataset: 35.657869389327686%
MAE: 7.297198999550411, MSE:91.50197843678657,
RMSE: 9.565666648842964, Index of Agreement: 0.8956910404724958

------------- Random set: 2 -----------------
Hidden: 2258, epochs: 94, learning rate: 0.002607420563510118


...

Accuracy on validation dataset: 41.14814101329224%
MAE: 5.9483259208561385, MSE:71.86781176838954,
RMSE: 8.477488529534531, Index of Agreement: 0.9106463528123214

------------- Random set: 3 -----------------
Hidden: 2182, epochs: 46, learning rate: 0.0007531111068484086


100%|██████████| 46/46 [00:12<00:00,  3.72it/s]


Accuracy on validation dataset: 40.820651126950494%
MAE: 6.167921388163327, MSE:80.1152416920237,
RMSE: 8.950711798065207, Index of Agreement: 0.8896575404887997

------------- Random set: 4 -----------------
Hidden: 2414, epochs: 40, learning rate: 0.002732658631120726


100%|██████████| 40/40 [00:10<00:00,  3.93it/s]

Accuracy on validation dataset: 39.45289924869967%
MAE: 6.48989212729842, MSE:79.38970769380634,
RMSE: 8.9100902180509, Index of Agreement: 0.8992715393221181


### SNN with LIF neurons

In [7]:
for i in range(5):
    h = np.random.randint(500, 2500)
    e = np.random.randint(20, 50)
    s = np.random.randint(20, 50)
    lr = np.random.uniform(0.0001, 0.005)
    print(f'\n------------- Random set: {i} -----------------\nHidden: {h}, epochs: {e}, steps: {s}, learning rate: {lr}')
    model = TwoLayerLeaky(num_inputs=8, num_hidden=h, num_outputs=1, beta=0.95, num_steps=s)
    training_loop(model, train_loader, test_loader, device, num_epochs=e, lr=lr)
    get_error_measures(model, test_loader, device=device, print_e=True)


------------- Random set: 0 -----------------
Hidden: 1448, epochs: 33, steps: 26, learning rate: 0.0003436980323828348


100%|██████████| 33/33 [01:19<00:00,  2.42s/it]


Accuracy on validation dataset: 33.01868618763244%
MAE: 7.894523208769638, MSE:120.92361301209081,
RMSE: 10.99652731602531, Index of Agreement: 0.7948390687522078

------------- Random set: 1 -----------------
Hidden: 1091, epochs: 32, steps: 40, learning rate: 0.0001328700997104934


100%|██████████| 32/32 [01:50<00:00,  3.46s/it]


Accuracy on validation dataset: 27.316509343093813%
MAE: 9.183800791160056, MSE:174.5187845653164,
RMSE: 13.210555800772214, Index of Agreement: 0.5198825337574162

------------- Random set: 2 -----------------
Hidden: 1933, epochs: 45, steps: 36, learning rate: 0.0042686513510183256


...

Accuracy on validation dataset: 38.99056058562898%
MAE: 6.643708319476611, MSE:90.93335691436572,
RMSE: 9.535898327602162, Index of Agreement: 0.8824818149234419

------------- Random set: 3 -----------------
Hidden: 632, epochs: 21, steps: 27, learning rate: 0.0036470198181153385


100%|██████████| 21/21 [00:49<00:00,  2.35s/it]


Accuracy on validation dataset: 37.892506260836065%
MAE: 7.2791851260911935, MSE:121.10956664736807,
RMSE: 11.004979175235547, Index of Agreement: 0.794129688283638

------------- Random set: 4 -----------------
Hidden: 2480, epochs: 49, steps: 31, learning rate: 0.0023397186853238348


100%|██████████| 49/49 [02:27<00:00,  3.01s/it]


Accuracy on validation dataset: 38.701598921209786%
MAE: 6.681780000753519, MSE:91.23994775688404,
RMSE: 9.551960414327732, Index of Agreement: 0.8709981509932454


### SNN with Synaptic neurons

In [8]:
for i in range(2):
    h = np.random.randint(1000, 3000)
    e = np.random.randint(10, 40)
    lr = np.random.uniform(0.0001, 0.005)
    print(f'\n------------- Random set: {i} -----------------\nHidden: {h}, epochs: {e}, learning rate: {lr}')
    model = TwoLayerSynaptic(num_inputs=8, num_hidden=h, num_outputs=1)
    training_loop(model, train_loader, test_loader, device, num_epochs=e, lr=lr)
    get_error_measures(model, test_loader, device=device, print_e=True)


------------- Random set: 0 -----------------
Hidden: 1071, epochs: 39, learning rate: 0.00320299681815868


100%|██████████| 39/39 [06:07<00:00,  9.43s/it]


Accuracy on validation dataset: 38.682334810248506%
MAE: 6.721668251675463, MSE:98.4209748683252,
RMSE: 9.920734593180345, Index of Agreement: 0.8528575223335096

------------- Random set: 1 -----------------
Hidden: 2795, epochs: 22, learning rate: 0.0032710867246137226


100%|██████████| 22/22 [03:27<00:00,  9.41s/it]


Accuracy on validation dataset: 39.626276247351186%
MAE: 6.711508377419113, MSE:91.21200452972431,
RMSE: 9.550497606393309, Index of Agreement: 0.8716763507631314


### SNN with additional layer

In [9]:
for i in range(5):
    h = np.random.randint(300, 2500)
    e = np.random.randint(10, 40)
    s = np.random.randint(20, 30)
    lr = np.random.uniform(0.0001, 0.005)
    print(f'\n------------- Random set: {i} -----------------\nHidden: {h}, epochs: {e}, steps: {s}, learning rate: {lr}')
    model = ThreeLayerLeaky(num_inputs=8, num_hidden=h, num_outputs=1, beta=0.95, num_steps=s)
    training_loop(model, train_loader, test_loader, device, num_epochs=e, lr=lr)
    get_error_measures(model, test_loader, device=device, print_e=True)


------------- Random set: 0 -----------------
Hidden: 798, epochs: 30, steps: 21, learning rate: 0.003061612357708661


100%|██████████| 30/30 [01:39<00:00,  3.33s/it]


Accuracy on validation dataset: 32.74898863417453%
MAE: 8.19164126299636, MSE:151.51189193265398,
RMSE: 12.309016692354186, Index of Agreement: 0.7116522039880617

------------- Random set: 1 -----------------
Hidden: 2487, epochs: 32, steps: 21, learning rate: 0.003881718827390492


100%|██████████| 32/32 [02:20<00:00,  4.40s/it]


Accuracy on validation dataset: 34.964361394721635%
MAE: 7.726730852590042, MSE:133.52276695483354,
RMSE: 11.555205188781095, Index of Agreement: 0.7732895056979301

------------- Random set: 2 -----------------
Hidden: 1802, epochs: 10, steps: 22, learning rate: 0.003421113742312809


...

Accuracy on validation dataset: 34.52128684261221%
MAE: 7.863482963583001, MSE:146.64537809764346,
RMSE: 12.109722461627412, Index of Agreement: 0.7010417131652835

------------- Random set: 3 -----------------
Hidden: 1719, epochs: 30, steps: 28, learning rate: 0.0025031815902070422


100%|██████████| 30/30 [02:10<00:00,  4.37s/it]


Accuracy on validation dataset: 33.230591408206514%
MAE: 8.192627550802653, MSE:145.77931417689152,
RMSE: 12.073910475769294, Index of Agreement: 0.7326767690825473

------------- Random set: 4 -----------------
Hidden: 1296, epochs: 28, steps: 27, learning rate: 0.00254261994175038


100%|██████████| 28/28 [01:54<00:00,  4.09s/it]


Accuracy on validation dataset: 29.994220766711617%
MAE: 9.072644862687053, MSE:162.19943239778,
RMSE: 12.735754096157008, Index of Agreement: 0.7178771300269294
